##### The Patstat library - Lesson 5
In this notebook we will make a study about the most influential inventors, using the applicant and CPC classification details.

## Example scenario
In lesson three we built a query with a double join, to display granted European patents filed this decade, the name of the inventor, and the amount of families that cite each application. Then we aggregated the results, summing the total citations for each inventor, as a proxy for the most influential inventors of the decade. 

We will then use this query as a filter for an outer query that finds the patent applications for an inventor in the list, based on their ranking. This could be useful e.g. if you want to write a news article about those influential inventors. 

## Subqueries in PATSTAT
As we have already seen, the PATSTAT library is an implementation of SQLAlchemy. 

In SQLAlchemy, when you create a subquery using the `subquery()` method, the resulting subquery object can be referenced in the outer query. This is particularly useful for nested queries where the result of one query is used as a filter or condition in another.

We will appli the `subquery()` method to the query of last example, limiting the query to the first entry to only get the first name in the list of top inventors. But first we need  to initalize our ORM client for PATSTAT.

In [8]:
# Importing the patstat client
from epo.tipdata.patstat import PatstatClient

# Initialize the PATSTAT client
patstat = PatstatClient()

# Access ORM
db = patstat.orm()

# Importing tables as models
from epo.tipdata.patstat.database.models import TLS201_APPLN, TLS207_PERS_APPLN, TLS206_PERSON

In [9]:
# Importing the func model
from sqlalchemy import func

# Defining the subquery for finding the top inventors
inner = db.query(
    TLS206_PERSON.person_id,  # inventor's name
    func.sum(TLS201_APPLN.nb_citing_docdb_fam).label('total_citations')  # sum of families citing patents by a given inventor
).join(
    TLS207_PERS_APPLN, TLS201_APPLN.appln_id == TLS207_PERS_APPLN.appln_id
).join(
    TLS206_PERSON, TLS207_PERS_APPLN.person_id == TLS206_PERSON.person_id
).filter(
    TLS201_APPLN.appln_filing_year >= 2020,
    TLS201_APPLN.appln_auth == 'EP',
    TLS201_APPLN.granted == 'Y',
    TLS207_PERS_APPLN.invt_seq_nr > 0  # filter to include only inventors
).group_by(
    TLS206_PERSON.person_id
).order_by(
    func.sum(TLS201_APPLN.nb_citing_docdb_fam).desc()  # order by total citations in descending order
).limit(1).subquery()


# Creating the outer query
outer_query = db.query(
    TLS201_APPLN.appln_id,
    TLS201_APPLN.appln_nr,
    TLS206_PERSON.person_name,
    TLS206_PERSON.person_id
).join(
    TLS207_PERS_APPLN, TLS201_APPLN.appln_id == TLS207_PERS_APPLN.appln_id
).join(
    TLS206_PERSON, TLS207_PERS_APPLN.person_id == TLS206_PERSON.person_id
).filter(
    TLS206_PERSON.person_id == inner.c.person_id,
    TLS201_APPLN.appln_auth == 'EP',
    TLS201_APPLN.granted == 'Y',
    TLS207_PERS_APPLN.invt_seq_nr > 0  
)

# Creating a dataframe with the results
patents_df = patstat.df(outer_query)

# Display the dataframe with detailed information about the patents of the selected inventor
patents_df

,appln_id,appln_nr,person_name,person_id
0,475146383,17155675,"HARRIS, Jason L.",53448894
1,473053132,16207245,"HARRIS, Jason L.",53448894
2,468445884,16186383,"HARRIS, Jason L.",53448894
3,545974287,21157430,"HARRIS, Jason L.",53448894
4,467919677,16185375,"HARRIS, Jason L.",53448894
...,...,...,...,...
102,511943162,19171920,"HARRIS, Jason L.",53448894
103,490213188,18157200,"HARRIS, Jason L.",53448894
104,507904622,19158219,"HARRIS, Jason L.",53448894
105,468445912,16186417,"HARRIS, Jason L.",53448894
